## 連結 mongoDB

In [11]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
from pymongo import Connection
connection = Connection()#這是在自己的本機建的，所以用connection

In [12]:
db = connection['Candidate']
db.collection_names(include_system_collections=False) #listing all of the collections in our database
# collection = db['comments']

[u'comments', u'posts', u'reactions', u'detector_one_by_one']

In [13]:
facebook_id = '46251501064' #Tsai
facebook_page_name = 'Tsai'
# facebook_id = '136845026417486' #Ke
# facebook_page_name = 'Ke'
# facebook_id = '188311137478' #ChinaTimes 
# facebook_page_name = 'ChinaTimes'
# facebook_id = '241284961029'
# facebook_page_name = 'udn' 
# facebook_id = '394896373929368' 
# facebook_page_name = 'ltn' 
# facebook_id = '232633627068' 
# facebook_page_name = 'AppleDaily'

In [16]:
# 匯入在2015-2017年間留言多於一則以上的使用者名單
file = open('user.txt', 'r') 
user_list = []
file = open('user.txt', 'r') 
userfile = file.read()
# print userfile.split(',')[0]
user_list.append(userfile.split(','))
print user_list[0][0:20]
print len(user_list[0])

['1307550719306500', '282520725500399', '10202745517877081', '1218878101553359', '357649311284175', '1326806630681931', '1077495905651938', '1745861102397361', '10206215950148794', '360637757653227', '167386673755045', '373679946346221', '10211808906327222', '1237124929644712', '1722221801352435', '1223024557715780', '1151289914941098', '384563838582738', '1423058981047363', '243217906134585']
43725


## 標記一篇，剩下自動標記

In [17]:
import string
import hanzidentifier
tsai_sup=['女性總統','小英一定要當選','支持蔡总统','蔡總統+U','台灣人民加油','小英健仁一定要凍蒜','小英姐加油','就是愛台灣',
          '蔡英文加油','英仁配當選','英仁配一定凍蒜','臺灣一定贏','超越黨派','把國民黨給消滅',
          '跳脫藍綠','2號蔡英文與陳建仁','蔡英文總統好','小英當選','回鄉投票','保障臺灣漁民權力',
          '總統英明','加油我的總統','高票當選','支持小英','有智慧的領導者','眾望所歸','總統加油',
          '支持您','支持你','支持妳','蔡主席加油','台灣加油','DPP完全執政','支持蔡英文',
          '國民黨不倒台灣不會好','民進黨加油','敬愛的總統','女領導人','蔡總統謝謝您','做小英後盾',
          '我挺小英','新政府請加油','新政府加油','我挺蔡英文','我愛小英','蔡總統加油','英英總統',
          '蔡英文總統加油','小英加油','女總統加油','阿英總統加油','蔡總統我支持你','小英總統我支持你',
          '英英總統我支持你','尤達大師','蔡英文總統我支持你','女總統我支持你','阿英總統我支持你']
tsai_not_sup=['台灣是中國的','金援外交','英雄的祖国','独立','总统府','解放军','被你骗','台湾的民主','军队','中华人民共和国万岁','蔡大妈','爛總統','两岸统一','女版陳水扁','日本鬼子','小日本','有同感的点赞',
              '大中华','喪權辱國','我们的教育让我们对你们产生了感情','難向政策','妳無恥的',
              '呆湾','湾湾','两岸一家亲','8.2趴的總統','空心蔡','中华民国和台湾','蔡英文八七分',
              '臺灣人系蠢材','支那','祖国统一','台湾人乡巴佬','人禽味','弱国无外交','妳跟馬英九一樣',
              '蔡英文的功力遠遠超越馬英九','空心','英文的英文','假愛台灣','喪犬','熱愛祖國','熱愛中國',
              '日杂绿畜','日畜','岛民思维','亲爱的祖国','狗日的','菜8趴','貼紙建國','老处女','热爱祖国',
              '台巴子','欢迎回家','茶叶蛋','不愿做奴隶的人们','罷免總統','罷免你','台独狗','台独分子',
              '蔡省长','蔡英文政府倒闭','民進黨太失望','民進黨失望','大陸爸爸','国民党','美国爸爸',
              '丢了邦交国','爱我中华','宣布独立','民進黨國民黨化','不想一例一休','擱置爭議','大陆在那头',
              '台湾独立','一群猴子','蔡后','真是無恥','轉型正義個屁','菜政府','民粹黨','蔡陰魂',
              '爛執政黨','快下台','下台啦','綠腦包','吃人夠夠','官逼民反','國民黨比民進黨更好',
              '餿掉的菜','蔡桶','菜桶','軍公教退休','菜老英','砍退休金','菜老板','你表姐',
              '總統會當嗎','蔡日狗','蔡英文你要不要臉','自自冉冉','台奸蔡英文','英英美代子',
              '白賊英','蔡英文下台','蔡英文爛死了','菜英文','空心菜','祭央乂','武則英','女版馬英九',
              '雙面蔡','蔡渣','18趴小英','亡國小英','菜豬']

#按照user去抓comment標記
for u in user_list[0]:
    for c in db.comments.find({'user_id':u,'page_name':'Tsai','year':{'$gte':2015,'$lte':2017}}):
#先找這個使用者有[沒有]在user_2中出現過
        if db.user.find({'user_id':c['user_id'].encode('utf-8')}).count()==0:
#                 偵測到簡體中文，自動判斷為不支持
            if hanzidentifier.is_simplified(c['comment'].encode('utf-8')) == True and hanzidentifier.identify(c['comment'].encode('utf-8'))is hanzidentifier.BOTH!= True:
                db.user.insert({'user_id':c['user_id'],
                               'user_name':c['user_name'],
                                'tsai_comment':[c['comment']],
                                'tsai_comment_id':[c['comment_id']],
                                'sup_page_name':'',
                                'sup_page_id':'',
                               'n_sup_page_name':facebook_page_name,
                                'n_sup_page_id':facebook_id,
                                'n_page_name':'',
                                'n_page_id':'',
                               })

            else: 
    #         把標點符號拿掉，看有沒有出現tsai_sup中的詞
    #                     [有]的話
                if any (ts in c['comment'].encode('utf-8').translate(None, string.punctuation) for ts in tsai_sup):
                    db.user.insert({'user_id':c['user_id'],
                                  'user_name':c['user_name'],
                                    'tsai_comment':[c['comment']],
                                    'tsai_comment_id':[c['comment_id']],
                                    'sup_page_name':facebook_page_name,
                                    'sup_page_id':facebook_id,
                                    'n_sup_page_name':'',
                                    'n_sup_page_id':'',
                                    'n_page_name':'',
                                    'n_page_id':'',
                                   })


        #               不然，如果[有]出現不支持裡面的詞
                elif any (no in c['comment'].encode('utf-8').translate(None, string.punctuation) for no in tsai_not_sup):
                    db.user.insert({'user_id':c['user_id'],
                                   'user_name':c['user_name'],
                                    'tsai_comment':[c['comment']],
                                    'tsai_comment_id':[c['comment_id']],
                                    'sup_page_name':'',
                                    'sup_page_id':'',
                                   'n_sup_page_name':facebook_page_name,
                                    'n_sup_page_id':facebook_id,
                                    'n_page_name':'',
                                    'n_page_id':'',
                                   })


                else:
                    pass 

    else:#在user中有找到，加上comment跟comment_id
        db.user.update({'user_id':c['user_id'].encode('utf-8')},
                       {'$push':{
                        'tsai_comment_id':[c['comment_id']],
                        'tsai_comment':[c['comment']]}})

db.user.count()
db.update
# comments.update()
print db.user.count()
client.close()
# db.posts.find({})

13631


## 把判斷不出來的comment id 與 comment content寫進去

In [19]:
#做comment id檔，把判斷不出來的id寫進去
file = open('net_id.txt','w') 
for u in user_list[0]:
    if db.user.find({'user_id':u}).count()==0:
        for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015,'$lte':2017},'user_id':u}):
            file.write(uu['user_id'].encode('utf-8')+',') 

file.close() 

In [20]:
#做comment文字檔
file = open('net.txt','w') 
for u in user_list[0]:
    if db.user_3.find({'user_id':u}).count()==0:
        for uu in db.comments.find({'page_name':'Tsai','year':{'$gte':2015,'$lte':2017},'user_id':u}):
            file.write(uu['comment'].encode('utf-8')+'\n') 

file.close() 

## 把reaction寫進去（包含按讚部隊）

In [21]:
# 寫太久了會斷掉
# 把reactions寫進去
for c in db.comments.find({'page_name':'Tsai','year':{'$gte':2015,'$lte':2017}}):
    for rea in db.reactions.find({'Post_ID':facebook_id+'_'+c['post_id']}):
        for u in rea[u'LOVE']:
            if db.user.find({'user_id':u}).count()!=0:
                db.user.update({'user_id':c['user_id'].encode('utf-8')},
                   {'$push':{
                    'LOVE':1}})
            else:
                db.user.insert({'user_id':u,
                                  'LOVE':1})
                
        for u in rea[u'ANGRY']:
            if db.user.find({'user_id':u}).count()!=0:
                db.user.update({'user_id':c['user_id'].encode('utf-8')},
                   {'$push':{
                    'ANGRY':1}})
            else:
                db.user.insert({'user_id':u,
                                  'ANGRY':1})

        for u in rea[u'HAHA']:
            if db.user.find({'user_id':u}).count()!=0:
                db.user.update({'user_id':c['user_id'].encode('utf-8')},
                   {'$push':{
                    'HAHA':1}})
            else:
                db.user.insert({'user_id':u,
                                  'HAHA':1})
        
        for u in rea[u'LIKE']:
            if db.user.find({'user_id':u}).count()!=0:
                db.user.update({'user_id':c['user_id'].encode('utf-8')},
                   {'$push':{
                    'LIKE':1}})
            else:
                db.user.insert({'user_id':u,
                                  'LIKE':1})
                
        for u in rea[u'SAD']:
            if db.user.find({'user_id':u}).count()!=0:
                db.user.update({'user_id':c['user_id'].encode('utf-8')},
                   {'$push':{
                    'SAD':1}})
            else:
                db.user.insert({'user_id':u,
                                  'SAD':1})
                
        for u in rea[u'WOW']:
            if db.user.find({'user_id':u}).count()!=0:
                db.user.update({'user_id':c['user_id'].encode('utf-8')},
                   {'$push':{
                    'WOW':1}})
            else:
                db.user.insert({'user_id':u,
                                  'WOW':1})

db.user.count()
db.update
        

CursorNotFound: cursor id '16598165436' not valid at server